# 🧟 Zombie Initiative Detection & Strategic Risk Analysis | Consulting Simulation

This notebook analyzes underperforming "Zombie" initiatives — projects that consume resources but fail to deliver measurable results.

It provides actionable insights by detecting financial waste, assessing initiative efficiency, and highlighting projects at risk of underperformance.

---

##  Objectives:
- Detect Zombie initiatives based on:
   - Zero team activity
   - No positive KPI outcomes
   - Budget overspend
- Quantify wasted budget and average loss per Zombie initiative
- Support strategic decision-making for resource optimization

---

##  Tools & Techniques:
- Python (Pandas) for data preparation and business rule application
- Cleaned project datasets aligned with consulting risk analysis standards
- Rule-based risk scoring (no automated ML applied)

---

In [31]:
from google.colab import files

# Upload files from your local machine to Colab
uploaded = files.upload()

In [32]:
import pandas as pd

# Load the uploaded datasets into pandas DataFrames
projects = pd.read_csv('projects_complex_final.csv')
activity = pd.read_csv('activity_logs_complex_final.csv')
kpi = pd.read_csv('kpi_data_complex_final.csv')
sentiment = pd.read_csv('sentiment_data_complex_final.csv')

In [33]:
# Display the first few rows of each DataFrame to inspect the data

projects.head()
activity.head()
kpi.head()
sentiment.head()

,project_id,employee_sentiment
0,1,Negative
1,2,Negative
2,3,Neutral
3,4,Negative
4,5,Negative


In [34]:
# Merge the DataFrames based on 'project_id' to create a comprehensive dataset

merged_df = pd.merge(projects, activity, on='project_id', how='left')
merged_df = pd.merge(merged_df, kpi, on='project_id', how='left')
merged_df = pd.merge(merged_df, sentiment, on='project_id', how='left')

merged_df.head()

,project_id,project_name,department,owner,budget_usd,target_budget,start_date,status,termination_reason,hours_logged,last_activity_date,kpi_name,kpi_value,last_updated,employee_sentiment
0,1,Google Glass,R&D,Sundar Team,5000000,4500000,2014-01-01,Terminated,Low Adoption,0,2015-01-01,Product Adoption,-5,2015-01-01,Negative
1,2,Google Plus,Product,Larry Team,3000000,2700000,2014-01-01,Terminated,Poor Engagement,0,2015-01-01,User Engagement,0,2015-01-01,Negative
2,3,Google Stadia,Gaming,Pichai Team,7000000,6300000,2014-01-01,Terminated,Market Saturation,0,2015-01-01,Market Share,-10,2015-01-01,Neutral
3,4,Amazon Dash,Product,Bezos Unit,4000000,3600000,2014-01-01,Terminated,Low Adoption,0,2015-01-01,Customer Usage,-5,2015-01-01,Negative
4,5,Meta Portal,Product,Zuckerberg Lab,3500000,3150000,2014-01-01,Terminated,Poor Engagement,0,2015-01-01,User Retention,0,2015-01-01,Negative


In [35]:
# Display information about the merged DataFrame

merged_df.info()

merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   project_id          250 non-null    int64 
 1   project_name        250 non-null    object
 2   department          250 non-null    object
 3   owner               250 non-null    object
 4   budget_usd          250 non-null    int64 
 5   target_budget       250 non-null    int64 
 6   start_date          250 non-null    object
 7   status              250 non-null    object
 8   termination_reason  55 non-null     object
 9   hours_logged        250 non-null    int64 
 10  last_activity_date  250 non-null    object
 11  kpi_name            250 non-null    object
 12  kpi_value           250 non-null    int64 
 13  last_updated        250 non-null    object
 14  employee_sentiment  250 non-null    object
dtypes: int64(5), object(10)
memory usage: 29.4+ KB


,project_id,project_name,department,owner,budget_usd,target_budget,start_date,status,termination_reason,hours_logged,last_activity_date,kpi_name,kpi_value,last_updated,employee_sentiment
0,1,Google Glass,R&D,Sundar Team,5000000,4500000,2014-01-01,Terminated,Low Adoption,0,2015-01-01,Product Adoption,-5,2015-01-01,Negative
1,2,Google Plus,Product,Larry Team,3000000,2700000,2014-01-01,Terminated,Poor Engagement,0,2015-01-01,User Engagement,0,2015-01-01,Negative
2,3,Google Stadia,Gaming,Pichai Team,7000000,6300000,2014-01-01,Terminated,Market Saturation,0,2015-01-01,Market Share,-10,2015-01-01,Neutral
3,4,Amazon Dash,Product,Bezos Unit,4000000,3600000,2014-01-01,Terminated,Low Adoption,0,2015-01-01,Customer Usage,-5,2015-01-01,Negative
4,5,Meta Portal,Product,Zuckerberg Lab,3500000,3150000,2014-01-01,Terminated,Poor Engagement,0,2015-01-01,User Retention,0,2015-01-01,Negative


In [36]:
# Identify "zombie" projects based on specific criteria:
# 1. Project status is 'Active'
# 2. No hours have been logged (hours_logged == 0)
# 3. The KPI value is less than or equal to 0 (no positive impact)

zombies = merged_df[
    (merged_df['status'] == 'Active') &
    (merged_df['hours_logged'] == 0) &
    (merged_df['kpi_value'] <= 0)
]

zombies.head()

,project_id,project_name,department,owner,budget_usd,target_budget,start_date,status,termination_reason,hours_logged,last_activity_date,kpi_name,kpi_value,last_updated,employee_sentiment
8,9,Project_9,Compliance,Pichai Team,2708513,2041754,2021-10-10,Active,NaN,0,2022-12-30,Revenue Impact,-10,2022-12-08 00:00:00,Neutral
29,30,Project_30,Compliance,Mohit Rana,5361415,3829612,2023-05-07,Active,NaN,0,2021-08-13,Customer Growth,-5,2022-07-30 00:00:00,Positive
33,34,Project_34,Product,Sundar Team,9213685,8070128,2023-12-07,Active,NaN,0,2021-07-16,Product Adoption,-15,2022-02-21 00:00:00,Positive
83,84,Project_84,Operations,Larry Team,2044841,1927357,2022-07-14,Active,NaN,0,2023-06-23,Healthcare Impact,-15,2022-11-04 00:00:00,Negative
157,158,Project_158,R&D,Mohit Rana,9210543,7537447,2018-05-30,Active,NaN,0,2021-07-29,Product Adoption,0,2023-01-01 00:00:00,Negative


In [37]:
# Calculate key metrics for the identified zombie projects:
# - Total budget wasted across all zombie projects
# - Total count of zombie projects
# - Average wasted budget per zombie project

total_wasted_budget = zombies['budget_usd'].sum()
total_zombie_count = zombies.shape[0]
average_wasted_budget = zombies['budget_usd'].mean()


print(f"Total Zombie Projects: {total_zombie_count}")
print(f"Total Wasted Budget: ${total_wasted_budget:,.2f}")
print(f"Average Wasted Budget per Zombie Project: ${average_wasted_budget:,.2f}")

Total Zombie Projects: 8
Total Wasted Budget: $42,503,060.00
Average Wasted Budget per Zombie Project: $5,312,882.50


In [38]:
#Department-Wise Zombie Waste Breakdown
dept_summary = zombies.groupby('department').agg(
    zombie_count = ('project_id', 'count'),
    total_wasted_budget = ('budget_usd', 'sum')
).reset_index()


print(dept_summary)

dept_summary_sorted = dept_summary.sort_values(by='zombie_count', ascending=False)
print(dept_summary_sorted)

   department  zombie_count  total_wasted_budget
0  Compliance             2              8069928
1       Legal             1              9349336
2  Operations             1              2044841
3     Product             2             12759750
4         R&D             1              9210543
5       Sales             1              1068662
   department  zombie_count  total_wasted_budget
0  Compliance             2              8069928
3     Product             2             12759750
1       Legal             1              9349336
2  Operations             1              2044841
4         R&D             1              9210543
5       Sales             1              1068662


In [39]:
# Analyze the monthly trend of zombie projects based on their start date

merged_df['start_date'] = pd.to_datetime(merged_df['start_date'])

zombies = merged_df[
    (merged_df['status'] == 'Active') &
    (merged_df['hours_logged'] == 0) &
    (merged_df['kpi_value'] <= 0)
]

# Extract the month and year from the 'start_date' and store it in a new column 'project_month'

zombies['project_month'] = zombies['start_date'].dt.to_period('M')

# Group the 'zombies' DataFrame by 'project_month' and count the number of zombie projects per month
monthly_trend = zombies.groupby('project_month').agg(
    zombie_count=('project_id', 'count')
).reset_index()


print(monthly_trend)

  project_month  zombie_count
0       2018-05             1
1       2020-01             1
2       2021-06             1
3       2021-10             1
4       2022-07             1
5       2023-05             1
6       2023-06             1
7       2023-12             1


/tmp/ipython-input-39-1571475085.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zombies['project_month'] = zombies['start_date'].dt.to_period('M')


In [40]:
# Analyze which departments have the most zombie projects and the highest wasted budget

zombies = merged_df[
    (merged_df['status'] == 'Active') &
    (merged_df['hours_logged'] == 0) &
    (merged_df['kpi_value'] <= 0)
]

dept_analysis = zombies.groupby('department').agg(
    zombie_count=('project_id', 'count'),
    total_wasted_budget=('budget_usd', 'sum')
).reset_index()

dept_analysis = dept_analysis.sort_values(by='zombie_count', ascending=False)

print(dept_analysis)

   department  zombie_count  total_wasted_budget
0  Compliance             2              8069928
3     Product             2             12759750
1       Legal             1              9349336
2  Operations             1              2044841
4         R&D             1              9210543
5       Sales             1              1068662


In [41]:
# Section for Predictive Zombie Risk modeling

In [42]:
import pandas as pd

# a new DataFrame 'model_df' with relevant columns for predictive modeling.
model_df = merged_df[['budget_usd', 'employee_sentiment', 'kpi_value', 'status', 'hours_logged']].copy()

# a binary target variable 'zombie_flag':
# 1 if the project is a zombie (Active, 0 hours_logged, kpi_value <= 0), 0 otherwise.
model_df['zombie_flag'] = ((model_df['status'] == 'Active') &
                           (model_df['hours_logged'] == 0) &
                           (model_df['kpi_value'] <= 0)).astype(int)

# Display the head of the 'model_df' to inspect the new column
model_df.head()

,budget_usd,employee_sentiment,kpi_value,status,hours_logged,zombie_flag
0,5000000,Negative,-5,Terminated,0,0
1,3000000,Negative,0,Terminated,0,0
2,7000000,Neutral,-10,Terminated,0,0
3,4000000,Negative,-5,Terminated,0,0
4,3500000,Negative,0,Terminated,0,0


In [43]:
# Encode the 'employee_sentiment' categorical variable into numerical representation
# Using a dictionary to map sentiment levels to numbers

sentiment_mapping = {
    'Negative': 0,
    'Neutral': 1,
    'Positive': 2
}

# Apply the mapping
model_df['sentiment_encoded'] = model_df['employee_sentiment'].map(sentiment_mapping)

model_df.drop('employee_sentiment', axis=1, inplace=True)

model_df.head()

,budget_usd,kpi_value,status,hours_logged,zombie_flag,sentiment_encoded
0,5000000,-5,Terminated,0,0,0
1,3000000,0,Terminated,0,0,0
2,7000000,-10,Terminated,0,0,1
3,4000000,-5,Terminated,0,0,0
4,3500000,0,Terminated,0,0,0


In [44]:

# Step 1: Map sentiment to numerical values for risk scoring.
# Here, 'Negative' sentiment is mapped to 1 (indicating higher risk), others to 0.

merged_df['sentiment_encoded'] = merged_df['employee_sentiment'].map({
    'Negative': 1,
    'Neutral': 0,
    'Positive': 0
})

# Calculating a 'total_risk_score' based on multiple factors:

merged_df['total_risk_score'] = (
    (merged_df['budget_usd'] > 1000000).astype(int) +
    (merged_df['sentiment_encoded'] == 1).astype(int) +
    (merged_df['kpi_value'] <= 0).astype(int)
)

model_df = merged_df[['project_id', 'budget_usd', 'sentiment_encoded', 'kpi_value', 'total_risk_score']]

model_df.head()

,project_id,budget_usd,sentiment_encoded,kpi_value,total_risk_score
0,1,5000000,1,-5,3
1,2,3000000,1,0,3
2,3,7000000,0,-10,2
3,4,4000000,1,-5,3
4,5,3500000,1,0,3


In [50]:
# Preparing data for machine learning model by splitting into training and testing sets

from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y) for the model

X = model_df[['budget_usd', 'sentiment_encoded', 'kpi_value']]
y = model_df['total_risk_score']

# Split the data into training (70%) and testing (30%) sets

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(X_train.shape, X_test.shape)

(175, 3) (75, 3)


In [49]:
# Training a simple predictive model (Decision Tree Classifier)
# Import the DecisionTreeClassifier model from scikit-learn

from sklearn.tree import DecisionTreeClassifier

# Initialize a Decision Tree Classifier model

clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train, y_train)


DecisionTreeClassifier(random_state=42)

In [47]:
# Evaluating the performance of the trained Decision Tree model

from sklearn.metrics import classification_report, confusion_matrix

# Using the trained model to make predictions on the test set features
y_pred = clf.predict(X_test)

# Evaluating the model using standard classification metrics:

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[ 1  0  0  0]
 [ 0 37  1  0]
 [ 0  0 30  0]
 [ 0  0  1  5]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      0.97      0.99        38
           2       0.94      1.00      0.97        30
           3       1.00      0.83      0.91         6

    accuracy                           0.97        75
   macro avg       0.98      0.95      0.97        75
weighted avg       0.97      0.97      0.97        75



In [48]:
# Save the processed DataFrames to CSV files

merged_df.to_csv('clean_zombie_dataset.csv', index=False)

model_df.to_csv('risk_scored_dataset.csv', index=False)